In [ ]:
# Inspect GPU configuration
# to inspect full configuration run nvidia-smi on the host
# running this container
if not torch.cuda.is_available():
    print('Not connected to a GPU')
else:
    print("Detected %s gpus" % torch.cuda.device_count())
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

# Inspect software configuration
pytorchv = !pip3 show torch | grep Version
print("Pytorch version: %s" % pytorchv)
tensorflowv = !pip3 show tensorflow | grep Version
print("Tensorflow version: %s" % tensorflowv)

In [ ]:
# Git repo clone
#!git clone https://github.com/MaxPayne86/Automated-GuitarAmpModelling.git
#!cd Automated-GuitarAmpModelling && git checkout feature/add-custom-bounds && git submodule update --init --recursive

In [ ]:
# Utils
class StopExecution(Exception):
  def _render_traceback_(self):
    pass

def parse_stats(stats):
  with open(stats) as json_file:
    data = json.load(json_file)
    test_lossESR_final = data['test_lossESR_final']
    test_lossESR_best = data['test_lossESR_best']
    esr = min(test_lossESR_final, test_lossESR_best)
    valuedict = {}
    valuedict["model_name"] = model_name
    valuedict["unit_type"] = unit_type
    valuedict["hidden_size"] = hidden_size
    valuedict["skip"] = skip
    return [esr, valuedict]

def write_results_dict(resultsdict, outfile):
  orderedlist = sorted(resultsdict)
  counter = 1
  for item in orderedlist:
    print("%d) %s: %s %d | skip = %d | ESR = %.6f" % (counter, maindict[item]['model_name'], maindict[item]['unit_type'], maindict[item]['hidden_size'], maindict[item]['skip'], item))
    counter = counter + 1
  with open(outfile, 'w') as fp:
    counter = 1
    for item in orderedlist:
      line = "%d) %s: %s %d | skip = %d | ESR = %.6f" % (counter, maindict[item]['model_name'], maindict[item]['unit_type'], maindict[item]['hidden_size'], maindict[item]['skip'], item)
      fp.write("%s\n" % line)
      counter = counter + 1

# Do not use tensorflow for training in this container (installation is broken)
# it is just there for the modelToKeras.py script to generate an RTNeural compatible model file
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Logs all tensorflow messages except INFO, WARNING, and ERROR

# Pre-defined network types list
types_list_full = ["GRU-8", "GRU-12", "LSTM-8", "LSTM-12", "LSTM-16", "LSTM-20"]
types_list_heavy = ["LSTM-16", "LSTM-20", "LSTM-40"]
types_list_light = ["GRU-12", "LSTM-12", "LSTM-16"]
# A model is defined by name and input/output track(s) plus network type and config
model_list = [
  {
    "name": "US Double Nrm",
    "input": "Recordings/Export/in-gtr-x-org.wav",
    "target": "Recordings/Export/US Double Nrm.wav",
    "csv": "yes",
    "types_list": ["LSTM-12"],
    "skip": [1]
  },
  {
    "name": "Cali Texas Ch 1",
    "input": "Recordings/Export/in-gtr-x-org.wav",
    "target": "Recordings/Export/Cali Texas Ch 1.wav",
    "csv": "yes",
    "types_list": ["LSTM-12"],
    "skip": [1]
  },
  {
    "name": "US Deluxe Nrm",
    "input": "Recordings/Export/in-gtr-x-org.wav",
    "target": "Recordings/Export/US Deluxe Nrm.wav",
    "csv": "yes",
    "types_list": types_list_light,
    "skip": [0, 1]
  },
  {
    "name": "US Small Tweed",
    "input": "Recordings/Export/in-gtr-x-org.wav",
    "target": "Recordings/Export/US Small Tweed.wav",
    "csv": "yes",
    "types_list": types_list_light,
    "skip": [0, 1]
  }
]

# Print model_list 
#for model in model_list:
#  print("%s\n %s\n %s\n %s" % (model['name'], model['input'], model['target'], model['csv']))
#  for type in model['types_list']:
#    print("  %s" % type)
#  for value in model['skip']:
#    print("  %d" % int(value))
#raise StopExecution

# Perform the training for each model
import json
!export CUBLAS_WORKSPACE_CONFIG=:4096:2
for model in model_list:
  resultsdict = {}
  for skip in model['skip']:
    for type in model['types_list']:
      # Write config file under ./Configs on-the-fly
      types_args = type.split('-')
      unit_type = types_args[0]
      hidden_size = int(types_args[1])
      loss_functions = {"ESR": 0.75, "DC": 0.25}
      pre_filt = "A-Weighting"
      device = "aidadsp-auto"
      file_name = "aidadsp-auto"
      samplerate = "48000"
      author = "Aida DSP"
      model_name = "RNN-" + device
      config = "Configs/RNN-%s.json" % device
      # Create config file on the fly
      config_data = {}
      config_data['hidden_size'] = hidden_size
      config_data['unit_type'] = unit_type
      config_data['loss_functions'] = loss_functions
      config_data['pre_filt'] = pre_filt
      config_data['skip'] = skip
      config_data['device'] = device
      config_data['file_name'] = file_name
      config_data['samplerate'] = samplerate
      config_data['author'] = author
      json_string = json.dumps(config_data)
      json_file = open(config, "w")
      json_file.write(json_string)
      json_file.close()
      input_file = model['input']
      target_file = model['target']
      # Create dataset on-the-fly
      if model['csv'] == "yes":
        !python3 prep_wav.py -f "$input_file" "$target_file" -l $model_name -csv
      else:
        !python3 prep_wav.py -f "$input_file" "$target_file" -l $model_name
      # Perform training
      from time import time, gmtime, strftime
      start = time()
      !python3 dist_model_recnet.py -l $model_name -slen 24000 --seed 39 -lm False
      end = time()
      delta = end - start
      print(strftime("%H:%M:%S", gmtime(delta)))
      # Convert json model file to RTNural format
      !python3 modelToKeras.py -l $model_name
      # Copy meaningful data to an output dir for later analysis
      now = !date +"%Y-%m-%d-%H:%M:%S"
      result_path = "Results/Jupyter/%s_%s" % (now[0], model_name)
      !mkdir -p $result_path
      #!tar czf $result_path/Data.tar.gz /content/Automated-GuitarAmpModelling/Data
      !tar czf $result_path/TensorboardData.tar.gz -C TensorboardData .
      !rm -rf TensorboardData
      #!tar czf $result_path/Results.tar.gz -C Results/*$model_name .
      !cp Results/*$model_name/model*.json Results/*$model_name/training_stats.json $result_path/
      meaningful_name = model['name']
      !cp $result_path/model_keras.json $result_path/$meaningful_name.json 
      !cp $config $result_path/
      # Update dict with training results
      result_dir = "Results/" + device + "-" + model_name
      stats = result_dir + "/training_stats.json"
      [esr, entry] = parse_stats(stats)
      resultsdict[esr] = entry
      !rm -rf Results/*$model_name
  # Write sorted results on a file
  result_file = "Results/Jupyter/%s_%s.txt" % (now[0], model_name)
  write_results_dict(resultsdict, result_file)